# IMPORTS - INITIALIZATION

In [1]:
import ee, eemont
import xee, wxee
import xarray as xr
import geopandas as gpd
import geemap

In [27]:
ee.Authenticate()
ee.Initialize()
wxee.Initialize()

# Import of index and Geom data


In [28]:
years = range(1985, 2025)

In [30]:
fc_lia = ee.FeatureCollection('') # Change to EU
geom_lia=fc_lia.geometry()

dem_glo30 = ee.ImageCollection("COPERNICUS/DEM/GLO30")

Importing TS IC

In [32]:
gbr_ts_filled_smooth=ee.ImageCollection([
    ee.Image(f"projects/ee-roniritzganem/assets/EU_GL/ts_GBR_NDWI_TIR/GBR/GBR_TS_filled_smooth_{year}")
    for year in years
])
tir_ts_filled_smooth=ee.ImageCollection([
    ee.Image(f"projects/ee-roniritzganem/assets/EU_GL/ts_GBR_NDWI_TIR/TIR/TIR_TS_filled_smooth_{year}").set('year', year).set("system:time_start", ee.Date(f"{year}"))
    for year in years
])
ndwi_ts_filled_smooth=ee.ImageCollection([
    ee.Image(f"projects/ee-roniritzganem/assets/EU_GL/ts_GBR_NDWI_TIR/NDWI/NDWI_TS_filled_smooth_{year}")
    for year in years
])

# Parameters 

In [33]:
scale = 30 #Landsat 30m
projection = "EPSG:4326" #GEE le plus stable avec ce SRC

In [34]:
dem = ee.ImageCollection(dem_glo30).filterBounds(fc_lia).select("DEM")
dem_mosaic = dem.mosaic().clip(fc_lia)
SlopeThresh = 30

In [35]:
visWATER_1 = {"min": 0, "max": 1, "palette": ["#808080", "#FFA500"]}  # Orange

# FUNCTIONS

In [36]:
def build_band_image(band_name,collection):
    def select_band(img):
        return img.select([band_name]) \
                  .copyProperties(img, img.propertyNames())
    
    return collection.map(select_band)

In [37]:
def select_band(img):
    years = range(1984, 2025)
    for year in years:
        img.copyProperties(img, img.propertyNames()) \
                    .set("system:time_start", ee.Date(f"{year}"))
    return img
    


In [38]:
def convert_temp_to_deg(image):
    image_degC = image.select('TIR') \
                      .subtract(273.15) \
                      .rename('tempC') \
                      .copyProperties(image, ['system:time_start'])
    return image_degC

In [39]:
def set_year_from_property(image):
    """
    Prend une image, lit une propriété 'year', et définit 'system:time_start' basé sur cette année.
    """
    # Lire l'année depuis les propriétés
    year = ee.Number(image.get('year'))
    
    # Créer une date au 1er janvier de cette année
    date = ee.Date.fromYMD(year, 1, 1)
    
    # Ajouter cette date dans 'system:time_start'
    return image.set('system:time_start', date.millis())

In [40]:
# Merge to filter both on NDWI and TIR value
def join_ndwi_tir(image_ndwi):
    year = image_ndwi.get('year')
    
    matching_tir = tir_ts_filled_smooth.filter(ee.Filter.eq('year', year)).first()
    
    combined = image_ndwi.addBands(matching_tir)
    
    return combined

In [42]:
def get_slope(image):
    return ee.Terrain.slope(image)

In [43]:
slope = dem.map(get_slope).mosaic().clip(fc_lia) \
#need to map on single image for slope function to work fine
slope_mask = slope.lt(SlopeThresh)

In [44]:
def classifyGlacier(image, seuil):
    temp = image.select('GBR').gt(seuil)
    mask = temp.where(temp.gte(1), 1)
    
    return image.addBands(mask.rename('GLACIER').toInt())
   

def classifyWater(image, seuil):
    temp = image.select('NDWI').gt(seuil)
    temp2 = temp.where(temp.gte(1), 2)
    mask = temp2.updateMask(slope_mask).unmask(0)

    # Temperature mask
    Temp_convert= image.select('tempC')
    Temp_mask = Temp_convert.updateMask(Temp_convert.gt(1).And(Temp_convert.lt(20)))
    mask2 = mask.updateMask(Temp_mask).unmask(0)
    
    return image.addBands(mask2.rename('WATER').toInt())

In [45]:
def clip_to_geometry(image,geom):
    return image.clip(geom)

-------------------------------------------------------------------------------------

# Threshold and Classification

GBR>6 and
NDWI>0.3

In [46]:
glacier_ts=gbr_ts_filled_smooth.map(lambda image: classifyGlacier(image, 6)).select('GLACIER').map(lambda img: img.selfMask()) \
                                .map(set_year_from_property)
water_ts = ndwi_ts_filled_smooth.map(join_ndwi_tir) \
                                .map(lambda image: classifyWater(image, 0.3)).select('WATER') \
                                .map(lambda img: img.selfMask()) \
                                .map(set_year_from_property)             


-------------------------------------------------------------------------------------

# Batch export

TS

In [51]:
for year in range(1984, 2025):  # de 1984 à 2024 inclus
    image = water_ts.filterDate(f'{year}-01-01', f'{year + 1}-01-01').first()
    globals()[f'water_ts_DEF_{year}'] = image

In [54]:
for year in range(1984, 2025):  # de 1984 à 2024 inclus
    image = glacier_ts.filterDate(f'{year}-01-01', f'{year + 1}-01-01').first()
    globals()[f'glacier_ts_DEF_{year}'] = image

------------------

To Drive

In [53]:
for year in range(1985, 2025):
    
    image_var_name = f"water_ts_DEF_{year}"
    file_prefix = f"water_{year}"
    # Récupérer dynamiquement l’image depuis les variables globales
    image = globals().get(image_var_name)
    task = ee.batch.Export.image.toDrive(
            image=image,
            folder='GEE_tel',
            description=f"GEE_tel_{year}",
            fileNamePrefix=file_prefix,
            region=geom_lia,
            scale=30,
            crs=projection,
            maxPixels=1e13
        )
    task.start()
    print(f'{year} done')

Export lancé pour l'année 1984
Export lancé pour l'année 1985
Export lancé pour l'année 1986
Export lancé pour l'année 1987
Export lancé pour l'année 1988
Export lancé pour l'année 1989
Export lancé pour l'année 1990
Export lancé pour l'année 1991
Export lancé pour l'année 1992
Export lancé pour l'année 1993
Export lancé pour l'année 1994
Export lancé pour l'année 1995
Export lancé pour l'année 1996
Export lancé pour l'année 1997
Export lancé pour l'année 1998
Export lancé pour l'année 1999
Export lancé pour l'année 2000
Export lancé pour l'année 2001
Export lancé pour l'année 2002
Export lancé pour l'année 2003
Export lancé pour l'année 2004
Export lancé pour l'année 2005
Export lancé pour l'année 2006
Export lancé pour l'année 2007
Export lancé pour l'année 2008
Export lancé pour l'année 2009
Export lancé pour l'année 2010
Export lancé pour l'année 2011
Export lancé pour l'année 2012
Export lancé pour l'année 2013
Export lancé pour l'année 2017
Export lancé pour l'année 2018
Export l

In [55]:
for year in range(1985, 2025):           
    image_var_name = f"glacier_ts_DEF_{year}"
    file_prefix = f"glacier_{year}"
    
    image = globals().get(image_var_name)
    task = ee.batch.Export.image.toDrive(
            image=image,
            folder='GEE_tel',
            description=f"GEE_tel_{year}",
            fileNamePrefix=file_prefix,
            region=geom_lia,
            scale=30,
            crs=projection,
            maxPixels=1e13
        )
    task.start()
    print(f'{year} done')


Export lancé pour l'année 1984
Export lancé pour l'année 1985
Export lancé pour l'année 1986
Export lancé pour l'année 1987
Export lancé pour l'année 1988
Export lancé pour l'année 1989
Export lancé pour l'année 1990
Export lancé pour l'année 1991
Export lancé pour l'année 1992
Export lancé pour l'année 1993
Export lancé pour l'année 1994
Export lancé pour l'année 1995
Export lancé pour l'année 1996
Export lancé pour l'année 1997
Export lancé pour l'année 1998
Export lancé pour l'année 1999
Export lancé pour l'année 2000
Export lancé pour l'année 2001
Export lancé pour l'année 2002
Export lancé pour l'année 2003
Export lancé pour l'année 2004
Export lancé pour l'année 2005
Export lancé pour l'année 2006
Export lancé pour l'année 2007
Export lancé pour l'année 2008
Export lancé pour l'année 2009
Export lancé pour l'année 2010
Export lancé pour l'année 2011
Export lancé pour l'année 2012
Export lancé pour l'année 2013
Export lancé pour l'année 2017
Export lancé pour l'année 2018
Export l

------------------

To Asset

In [ ]:
for year in range(1985, 2025):
    
    image_var_name = f"water_ts_DEF_{year}"
    file_prefix = f"water_{year}"
    # Récupérer dynamiquement l’image depuis les variables globales
    image = globals().get(image_var_name)
    task = ee.batch.Export.image.toAsset(
            image=image,
            description=None,
            assetId=f'projects/ee-roniritzganem/assets/EU_GL/water_collection/water_{1984+i}',
            region=geom_lia,
            scale=30,
            crs=projection,
            maxPixels=1e13
        )
    task.start()
    print(f'{year} done')

In [ ]:
for year in range(1985, 2025):           
    image_var_name = f"glacier_ts_DEF_{year}"
    file_prefix = f"glacier_{year}"
    
    image = globals().get(image_var_name)
    task = ee.batch.Export.image.toDrive(
            image=image,
            description=None,
            assetId=f'projects/ee-roniritzganem/assets/EU_GL/glacier_collection/glacier_{1984+i}',
            region=geom_lia,
            scale=30,
            crs=projection,
            maxPixels=1e13
        )
    task.start()
    print(f'{year} done')